In [ ]:
import os

from spotmax import pipe

In [ ]:
work_dir = os.getcwd()
IMAGE_PATH = os.path.join()

In [ ]:
image = 

## Segment the spots
By segmenting the spots image we get a mask where to detect the spots. 

This is a **crucial step**. It is worth testing all the available methods which are several algortithms for automatic thresholding, the spotMAX neural network, and any of the models from the [BioImage.IO Model Zoo](https://bioimage.io).

Here we show how to visualize the result for all the methods and proceed with the selected one. 

### Segment with all the automatic thresholding methods

In [ ]:
result = pipe.spots_semantic_segmentation(
    image, 
    lab=None,
    gauss_sigma=0.0,
    spots_zyx_radii_pxl=None, 
    do_sharpen=False, 
    do_remove_hot_pixels=False,
    lineage_table=None,
    do_aggregate=True,
    use_gpu=False,
    logger_func=print,
    thresholding_method=None,
    keep_input_shape=True,
    nnet_model=None,
    nnet_params=None,
    nnet_input_data=None,
    bioimageio_model=None,
    bioimageio_params=None,
    do_preprocess=True,
    do_try_all_thresholds=True,
    return_only_segm=False,
    pre_aggregated=False,
    raw_image=None
)